<a href="https://colab.research.google.com/github/davidbonar1/03MAIR_Algoritmos_de_Optimizacion_2019/blob/master/David_Bo%C3%B1ar_alvarez_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 3 de Algoritmos de Optimización

David Boñar

https://colab.research.google.com/drive/1Kj2deB9jkmRiX4fAXHyzl8l_wq8Mxxju

https://github.com/davidbonar1/03MAIR_Algoritmos_de_Optimizacion_2019

Ejercicio 1: Problema del agente viajero

In [1]:
!pip install requests

In [2]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7fab8d892be0>)

In [3]:
!pip install tsplib95

In [0]:
import tsplib95
import random
import math

In [0]:
#Cargamos el problema en la variable problem
problem = tsplib95.load_problem(file)

In [0]:
#Vamos a ver el número de nodos que tenemos (deberían salir 42 ciudades/nodos) y las aristas
Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())

In [0]:
#Funciones base:
def crear_solucion_aleatoria(Nodos): #solucion aleatoria que comienza en el nodo 0
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))] #hacemos que tome un nodo no elegido anteriormente (es decir quitando el 0 y los elegido antes)
  return solucion

def distancia(a,b,problem): #distancia entre 2 nodos
  return problem.wfunc(a,b)

def distancia_total(solucion, problem): #distancia total de una trayectoria
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


In [8]:
solucion = crear_solucion_aleatoria(Nodos)
distancia_total (solucion, problem)

4828

In [9]:
#Resolveremos el problema por búsqueda aleatoria

def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion_aleatoria(Nodos)
  mejor_valor = distancia_total(mejor_solucion,problem)

  for i in range(N):
    solucion = crear_solucion_aleatoria(Nodos)
    valor = distancia_total(solucion,problem)
    if valor < mejor_valor:
      mejor_solucion = solucion
      mejor_valor = valor
  print("La mejor solución es: ",mejor_valor,mejor_solucion)

busqueda_aleatoria(problem,100000)

La mejor solución es:  3478 [0, 32, 28, 2, 38, 4, 6, 29, 10, 3, 1, 25, 22, 36, 16, 7, 14, 13, 5, 19, 18, 15, 37, 17, 26, 21, 24, 39, 41, 20, 35, 34, 8, 27, 11, 12, 9, 40, 23, 31, 30, 33]


La mejor solución encontrada es de 1273. Con este método d eresolución hemos obtenido del orden de 3500.

In [10]:
#Generador de vecindad

def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones. Para este problema 820.
  
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1): #cogemos desde el segundo nodo en adelante
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] #se intercamia el nodo i por el nodo j, y el j por el nodo i
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion
      
solucion = crear_solucion_aleatoria(Nodos)
mejor_solucion = genera_vecina(solucion)
print(mejor_solucion)

[0, 20, 3, 11, 30, 29, 37, 16, 26, 17, 38, 40, 9, 7, 13, 39, 21, 23, 10, 31, 5, 22, 18, 27, 15, 25, 19, 33, 32, 28, 1, 6, 12, 24, 14, 4, 34, 41, 8, 2, 36, 35]


In [11]:
#Búsqueda local
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion_aleatoria(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina
  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

busqueda_local(problem,1000)

La mejor solución encontrada es  1594 [0, 30, 29, 21, 24, 40, 23, 41, 9, 39, 22, 38, 34, 33, 20, 32, 31, 35, 36, 17, 37, 15, 16, 14, 7, 1, 6, 26, 18, 28, 8, 10, 25, 11, 12, 13, 19, 5, 4, 3, 2, 27]


En este caso, con 100 veces menos iteraciones que con la búsqueda aleatoria hemos obtenido un resultado mucho mejor (doble).

In [0]:
#Recocido simulado
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

#Funcion de probabilidad para determinar si se cambia a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False

def bajar_temperatura(T): #descanso exponencial
  return T*.9999


In [13]:
#Comprobación de que se intercambian los nodos
solucion = crear_solucion_aleatoria(Nodos)
print(solucion,"\n" ,genera_vecina_aleatorio(solucion))

[0, 30, 23, 17, 3, 1, 29, 35, 7, 33, 21, 15, 18, 32, 24, 8, 13, 39, 25, 11, 10, 27, 16, 22, 19, 14, 41, 36, 12, 5, 9, 4, 31, 26, 28, 38, 20, 37, 34, 2, 6, 40] 
 [0, 30, 23, 17, 3, 1, 29, 35, 7, 33, 21, 15, 18, 32, 24, 8, 6, 39, 25, 11, 10, 27, 16, 22, 19, 14, 41, 36, 12, 5, 9, 4, 31, 26, 28, 38, 20, 37, 34, 2, 13, 40]


In [0]:
def recocido_simulado(problem, TEMPERATURA):
  #problem = datos del problema

  solucion_referencia = crear_solucion_aleatoria(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1: #critario de parada
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion


In [15]:
sol = recocido_simulado(problem,100000)

	****Encontrada mejor solucion: 4692 [0, 33, 36, 2, 34, 20, 8, 29, 7, 19, 30, 18, 12, 5, 21, 41, 40, 25, 27, 10, 9, 1, 32, 38, 13, 17, 39, 15, 24, 6, 26, 28, 3, 37, 22, 16, 4, 14, 31, 35, 23, 11] 

	****Encontrada mejor solucion: 4611 [0, 13, 36, 2, 34, 20, 8, 29, 7, 19, 30, 18, 12, 5, 21, 41, 40, 25, 27, 10, 9, 1, 32, 38, 33, 17, 24, 15, 39, 6, 26, 28, 3, 37, 22, 16, 4, 14, 31, 35, 23, 11] 

	****Encontrada mejor solucion: 4601 [0, 13, 36, 2, 34, 20, 8, 29, 19, 7, 30, 18, 12, 5, 21, 41, 40, 25, 27, 10, 9, 1, 32, 38, 33, 17, 24, 15, 39, 6, 26, 28, 3, 37, 22, 16, 4, 14, 31, 35, 23, 11] 

	****Encontrada mejor solucion: 4528 [0, 13, 29, 2, 10, 7, 3, 28, 11, 19, 30, 18, 12, 5, 21, 41, 40, 1, 27, 34, 9, 25, 23, 38, 26, 17, 24, 15, 39, 6, 36, 32, 8, 37, 22, 4, 16, 14, 31, 35, 33, 20] 

	****Encontrada mejor solucion: 4494 [0, 13, 29, 2, 10, 7, 3, 28, 11, 19, 30, 18, 12, 5, 21, 41, 40, 1, 27, 34, 9, 25, 23, 38, 26, 14, 24, 15, 39, 6, 36, 32, 8, 37, 22, 4, 16, 17, 31, 35, 33, 20] 

	****Encon

In [0]:
from numpy.random import choice
#Colonia de hormigas
def probabilidad_de_preferencia(alpha, beta, Nodos, i,j,Nodos_no_visitados, T):
  sumatorio = 0
  for l in Nodos_no_visitados:
    sumatorio += (T[i][l]**alpha)*(1/distancia(i,l,problem)**beta)
    #print(sumatorio)
  if j not in Nodos_no_visitados:
    return 0
  probabilidad = ((T[i][j])**alpha*(1/distancia(i,j,problem)**beta))/sumatorio
  return probabilidad

def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  Nodos_no_visitados = list(set(range(1,len(Nodos))) - set(H) )  
  probabilidad = []
  for j in Nodos:
    probabilidad.append(probabilidad_de_preferencia(.8,.5,Nodos,H[-1],j,Nodos_no_visitados,T))
  #print(probabilidad)
  #print(choice(Nodos, p=probabilidad))
  return choice(Nodos, p=probabilidad)


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [0]:
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]

   #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)

In [0]:
hormigas(problem,5)